In [ ]:
import json
import os
import sys
import math
import argparse
import numpy as np
from scipy.special import erf
from scipy.interpolate import interp1d
from mpl_toolkits.mplot3d import Axes3D
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm
from matplotlib import patches as mpatches
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))

import shilofue.TwoDSubduction0.VtkPp as TwoDVtkPp
import shilofue.VtkPp as VtkPp
from shilofue.VtkPp import get_r
from shilofue.TwoDSubduction0.VtkPp import SlabPressures
from shilofue.TwoDSubduction0.PlotVisit import VISIT_OPTIONS

# import utilities in subdirectiory
sys.path.append(os.path.join(ASPECT_LAB_DIR, 'utilities', "python_scripts"))
import Utilities

In [ ]:
# start by presenting the vtu_snapshot (step + initial adaptive refinement)
case_dir= os.path.join(ASPECT_LAB_DIR, "tests/integration/big_fixtures/TwoDSubduction/eba_cdpt_coh500_SA80.0_OA40.0_cd100.0_cd7.5_gr9")
vtu_snapshot=105

In [ ]:
# initiate the class with options (using kwargs)
# we assume the imported data has the dynamic pressure as inputs
# To get a reasonable envelop that captures the shape of the slab, we need
# to use the dT method to derive the envelop.
# The "slab_envelop_interval" needs to be chosen to a large value, otherwise the points on the envelop
# will jump in and out.
kwargs = {"use_dT": 1, "output_slab": 1, "slab_envelop_interval": 10e3}
VTKP = TwoDVtkPp.VTKP

# setup
indent = kwargs.get("indent", 0)  # indentation for outputs
print("%s%s: Start" % (indent*" ", Utilities.func_name()))
use_dT = kwargs.get('use_dT', False)  # determine whether the slab envelop is by dT or by composition
dT = kwargs.get('dT', -100.0)
slab_envelop_interval = kwargs.get("slab_envelop_interval", 5e3)
ha_file = os.path.join(case_dir, "output", "depth_average.txt")
assert(os.path.isfile(ha_file))
output_path = os.path.join(case_dir, "vtk_outputs") # results are saved under the "vtk_outputs" directory
if not os.path.isdir(output_path):
    os.mkdir(output_path)

In [ ]:
# look for the input file named as "output/solution/solution-%05d.pvtu"
filein = os.path.join(case_dir, "output", "solution",\
     "solution-%05d.pvtu" % (vtu_snapshot))
assert(os.path.isfile(filein))
# get configurations with the VISIT_OPTIONS class
Visit_Options = VISIT_OPTIONS(case_dir)
Visit_Options.Interpret()
geometry = Visit_Options.options['GEOMETRY']
vtu_step = max(0, int(vtu_snapshot) - int(Visit_Options.options['INITIAL_ADAPTIVE_REFINEMENT']))
_time, step = Visit_Options.get_time_and_step(vtu_step)
# initiate class and read in the pvtu file
VtkP = VTKP(ha_file=ha_file, time=_time, slab_envelop_interval=slab_envelop_interval)
VtkP.ReadFile(filein)

In [ ]:
# Construct the poly data from the vtu files
# First, assign the fields to load.
# The dynamic pressure will be loaded automatically if it is present in the data
field_names = ['T', 'p', 'density', 'spcrust', 'spharz']
has_dynamic_pressure = int(Visit_Options.options['HAS_DYNAMIC_PRESSURE']) 
if has_dynamic_pressure == 1:
    field_names += ['nonadiabatic_pressure']
VtkP.ConstructPolyData(field_names, include_cell_center=True, construct_Tdiff=True)

In [ ]:
# include a v_profile
# This yield a vertical profile along a line segment in the z direction.
# the value of x1 is the lateral coordinate of the vertical profile.
# for the chunk geometry, x1 is the angle in radian.
# This profile is then used to derive the reference densities at depth
r0_range = [6371e3 - 2890e3, 6371e3]
Ro = 6371e3
x1 = 0.1  # Note: This cannot be very small (e.g. a 0.01 value will cause spikes in the density profile generated)
n = 100
v_profile = VtkP.VerticalProfile2D(r0_range, x1, n)

In [ ]:
# additional outputs
# output poly data, for debug usage
# The following variables define whether we output the slab envelop
output_slab = True
output_poly_data = False
if output_poly_data:
    file_out = os.path.join(output_path, "processed-%05d.vtp" % vtu_snapshot)
    VtkPp.ExportPolyData(VtkP.i_poly_data, file_out)
    file_out_1 = os.path.join(output_path, "processed_center-%05d.vtp" % vtu_snapshot)
    VtkPp.ExportPolyData(VtkP.c_poly_data, file_out_1)
# Derive the slab envelop.
#   dT method: using the difference in temperature to get the envelop of the slab
#   not-dT method: take the envelop of the crust and harzburgite boundary
if use_dT:
    VtkP.PrepareSlabByDT(slab_threshold=dT)  # slab: differential temperature
else:
    VtkP.PrepareSlab(['spcrust', 'spharz'])  # slab: composition
# output slab profile
if output_slab:
    stamp = "comp"
    if use_dT:
        stamp = "dT%.1f" % (abs(dT))
    slab_envelop0, slab_envelop1 = VtkP.ExportSlabEnvelopCoord()
    slab_internal = VtkP.ExportSlabInternal(output_xy=True)
    o_slab_env0 = os.path.join(case_dir,\
        "vtk_outputs", "slab_%s_env0_%.1fkm_%05d.vtp" % (stamp, slab_envelop_interval, vtu_step)) # envelop 0
    o_slab_env1 = os.path.join(case_dir,\
        "vtk_outputs", "slab_%s_env1_%.1fkm_%05d.vtp" % (stamp, slab_envelop_interval, vtu_step)) # envelop 1
    o_slab_in = os.path.join(case_dir,\
        "vtk_outputs", "slab_internal_%s_%.1fkm_%05d.txt" % (stamp, slab_envelop_interval, vtu_step)) # envelop 1
    VtkPp.ExportPolyDataFromRaw(slab_envelop0[:, 0], slab_envelop0[:, 1], None, None, o_slab_env0) # write the polydata
    VtkPp.ExportPolyDataFromRaw(slab_envelop1[:, 0], slab_envelop1[:, 1], None, None, o_slab_env1) # write the polydata
    np.savetxt(o_slab_in, slab_internal)
    print("%s%s: write file %s" % (indent*" ", Utilities.func_name(), o_slab_in))

In [ ]:
# Buoyancy forces
# the depth increment defines the unit bin width where the integral
# is computed upon. This is also used in calculaing the gradient, so
# the idea is to get a large enough value to average out the spikes
depth_increment = 20e3

# First, prepare the objects needed for computing the buoyancy
grav_acc = 10.0
assert(VtkP.include_cell_center)
assert(len(VtkP.slab_cells) > 0)
n_depth = int(np.ceil(VtkP.slab_depth / depth_increment))
buoyancies = np.zeros(n_depth) # save the values of buoyancy with ranges of depths
depths = []  # construct depths, each serve as the center in [depth - depth_increment/2.0, depth + depth_increment/2.0]
for i in range(n_depth):
    depth = (i + 0.5) * depth_increment
    depths.append(depth)
depths = np.array(depths)
centers = vtk_to_numpy(VtkP.c_poly_data.GetPoints().GetData())  # note these are data mapped to cell center
density_data = vtk_to_numpy(VtkP.c_poly_data.GetPointData().GetArray('density'))
density_ref_func = v_profile.GetFunction('density')

# Then, compute the slab buoyancy
# This is executed by looping the cells inside the slab and integrate their buoyancies
total_buoyancy = 0.0
density_profile = []
for i in VtkP.slab_cells:
    x = centers[i][0]
    y = centers[i][1]
    r = get_r(x, y, VtkP.geometry)
    i_r = int(np.floor((VtkP.Ro - r) / depth_increment))
    density = density_data[i]
    density_ref = density_ref_func(r)
    cell_size = VtkP.cell_sizes[i]
    buoyancy = - grav_acc * (density - density_ref) * cell_size  # gravity
    buoyancies[i_r] += buoyancy
    total_buoyancy += buoyancy
    density_profile.append([r, density])

# total buoyancy: (N / m); The meaning is the buoyancy forces related to a unit lateral length
# buoyancies: contains the "buoyancy gradient (N / m2)"; The meaning is the buoyancy forces of a unit lateral length in 
#   a unit depth increment
buoyancy_gradients = buoyancies / depth_increment
density_profile = np.array(density_profile)
b_profile = np.zeros((n_depth, 2))
b_profile[:, 0] = depths
b_profile[:, 1] = buoyancies

In [ ]:
# Generate plots
# 1. the reference density and the density vs depth
#    check if the curve of the reference density is smooth enough
#    check if the scatter points of density roughly increases with depth and continues to the depth of the subduction tip.
# 2. the boundary force with depth
fig, ax = plt.subplots()
radius = np.linspace(6371e3-2890e3, 6371e3, 100)
ref_densities = density_ref_func(radius)
ax.plot(ref_densities, radius/1e3)
ax.plot(density_profile[:, 1], density_profile[:, 0]/1e3, '.')
ax.grid()
# plot the buoyancy increments
fig, ax = plt.subplots()
ax.plot(buoyancy_gradients, depths/1e3, '.')
ax.invert_yaxis()
ax.set_xlabel("Buoyancy (N/m^2)")
ax.set_ylabel("Depth (km)")
ax.grid()

# Dump the buoyancies and depths to a file
# use this file to create polyfit in R
fname = os.path.join(ASPECT_LAB_DIR, "dtemp", "export_slab_buoyancy.txt")
np.savetxt(fname, np.concatenate((depths.reshape(len(depths), 1), buoyancy_gradients.reshape(len(buoyancies), 1)), axis=1))
print("File exported: ", fname)

In [ ]:
# The following two sections focus on pressure, and dip angle along the top and bottom envelops, respectively

slab_envelop = slab_envelop0
get_dip = TwoDVtkPp.get_dip

Ro = 6371e3
fileout = kwargs.get('fileout', None)
indent = kwargs.get('indent', 0)
has_dynamic_pressure = kwargs.get('has_dynamic_pressure', 0)
rs_n = 5 # resample interval
ip_interval = 1e3  # interval for interpolation
# resample the original envelop dataset
n_point = slab_envelop.shape[0]
rs_idx = range(0, n_point, rs_n)
slab_envelop_rs = slab_envelop[np.ix_(rs_idx, [0, 1])]
slab_env_polydata = VtkPp.InterpolateGrid(VtkP.i_poly_data, slab_envelop_rs, quiet=True) # note here VtkPp is module shilofue/VtkPp, while the VtkP is the class
temp_vtk_array = slab_env_polydata.GetPointData().GetArray('p')
env_ps  = vtk_to_numpy(temp_vtk_array)
# dynamic pressure is outputed -> read in
# dynamic pressure is not outputed -> use pressure - static_pressure as an estimation
if has_dynamic_pressure == 1:
    temp_vtk_array = slab_env_polydata.GetPointData().GetArray('nonadiabatic_pressure')
    env_dps  = vtk_to_numpy(temp_vtk_array)
    print("Read in the dynamic pressures")
else:
    env_dps = None
# import data onto selected points
depths = np.zeros(slab_envelop_rs.shape[0]) # data on envelop0
ps = np.zeros((slab_envelop_rs.shape[0], 4)) # pressure, horizontal & vertical components, dynamic pressure
thetas = np.zeros((slab_envelop_rs.shape[0], 1))
is_first = True
for i in range(0, slab_envelop_rs.shape[0]):
    x = slab_envelop_rs[i, 0]
    y = slab_envelop_rs[i, 1]
    theta_xy = np.arctan2(y, x)
    r = (x*x + y*y)**0.5
    depth = Ro - r  # depth of this point
    p = env_ps[i]  # pressure of this point
    p_static = VtkP.StaticPressure([r, VtkP.Ro], theta_xy, 2000)
    if env_dps is not None:
        p_d = env_dps[i]
    else:
        p_d = p - p_static # dynamic pressure, read in or compute
    depths[i] = depth
    d1 = 0.0  # coordinate differences
    d2 = 0.0
    # here we first get a dip angle
    if is_first:
        xnext = slab_envelop_rs[i+1, 0]  # coordinates of this and the last point
        ynext = slab_envelop_rs[i+1, 1]
        theta = get_dip(x, y, xnext, ynext, VtkP.geometry)
        is_first = False
    else: 
        xlast = slab_envelop_rs[i-1, 0]  # coordinates of this and the last point
        ylast = slab_envelop_rs[i-1, 1]
        theta = get_dip(xlast, ylast, x, y, VtkP.geometry) 
    thetas[i, 0] = theta
    # then we project the pressure into vertical and horizontal
    p_v = p * np.cos(theta)
    p_h = p * np.sin(theta)
    ps[i, 0] = p
    ps[i, 1] = p_h
    ps[i, 2] = p_v
    ps[i, 3] = p_d

depths0 = depths.copy()
ps0 = ps.copy()

In [ ]:
# top boundary, derive the theta value on this boundary
slab_envelop = slab_envelop1
get_dip = TwoDVtkPp.get_dip

Ro = 6371e3
fileout = kwargs.get('fileout', None)
indent = kwargs.get('indent', 0)
has_dynamic_pressure = kwargs.get('has_dynamic_pressure', 0)
rs_n = 1# 5 # resample interval
ip_interval = 1e3  # interval for interpolation
# resample the original envelop dataset
n_point = slab_envelop.shape[0]
rs_idx = range(0, n_point, rs_n)
slab_envelop_rs = slab_envelop[np.ix_(rs_idx, [0, 1])]
slab_env_polydata = VtkPp.InterpolateGrid(VtkP.i_poly_data, slab_envelop_rs, quiet=True) # note here VtkPp is module shilofue/VtkPp, while the VtkP is the class
temp_vtk_array = slab_env_polydata.GetPointData().GetArray('p')
env_ps  = vtk_to_numpy(temp_vtk_array)
# dynamic pressure is outputed -> read in
# dynamic pressure is not outputed -> use pressure - static_pressure as an estimation
if has_dynamic_pressure == 1:
    temp_vtk_array = slab_env_polydata.GetPointData().GetArray('nonadiabatic_pressure')
    env_dps  = vtk_to_numpy(temp_vtk_array)
    print("Read in the dynamic pressures")
else:
    env_dps = None
# import data onto selected points
depths = np.zeros(slab_envelop_rs.shape[0]) # data on envelop0
ps = np.zeros((slab_envelop_rs.shape[0], 4)) # pressure, horizontal & vertical components, dynamic pressure
thetas = np.zeros((slab_envelop_rs.shape[0], 1))
is_first = True
for i in range(0, slab_envelop_rs.shape[0]):
    x = slab_envelop_rs[i, 0]
    y = slab_envelop_rs[i, 1]
    theta_xy = np.arctan2(y, x)
    r = (x*x + y*y)**0.5
    depth = Ro - r  # depth of this point
    p = env_ps[i]  # pressure of this point
    p_static = VtkP.StaticPressure([r, VtkP.Ro], theta_xy, 2000)
    if env_dps is not None:
        p_d = env_dps[i]
    else:
        p_d = p - p_static # dynamic pressure, read in or compute
    depths[i] = depth
    d1 = 0.0  # coordinate differences
    d2 = 0.0
    # here we first get a dip angle
    if is_first:
        xnext = slab_envelop_rs[i+1, 0]  # coordinates of this and the last point
        ynext = slab_envelop_rs[i+1, 1]
        rnext = (xnext*xnext + ynext*ynext)**0.5
        theta_xy_next = np.arctan2(ynext, xnext)
        theta = get_dip(x, y, xnext, ynext, VtkP.geometry)
        is_first = False
    else: 
        xlast = slab_envelop_rs[i-1, 0]  # coordinates of this and the last point
        ylast = slab_envelop_rs[i-1, 1]
        rlast = (xlast*xlast + ylast*ylast)**0.5
        theta_xy_last = np.arctan2(ylast, xlast)
        theta = get_dip(xlast, ylast, x, y, VtkP.geometry)
    thetas[i, 0] = theta
    # then we project the pressure into vertical and horizontal
    p_v = p * np.cos(theta)
    p_h = p * np.sin(theta)
    ps[i, 0] = p
    ps[i, 1] = p_h
    ps[i, 2] = p_v
    ps[i, 3] = p_d

depths1 = depths.copy()
ps1 = ps.copy()

In [ ]:
# plot the envelop of the slab surface (top)
fig, ax = plt.subplots()
ax.plot(slab_envelop_rs[:, 0], slab_envelop_rs[:, 1], 'b*')
ax.set_aspect('equal')

In [ ]:
# plot the static pressure, dynamic pressure and the dip angle along depth
fig, ax = plt.subplots()
ax.plot(ps0[:, 0], depths0/1e3, "b*", label="lower slab envelop")
ax.plot(ps1[:, 0], depths1/1e3, "y*", label="upper slab envelop")
ax.invert_yaxis()
ax.set_xlabel("Static Pressure (Pa)")
ax.set_ylabel("Depth (km)")
ax.legend()

fig, ax = plt.subplots()
ax.plot(ps0[:, 3], depths0/1e3, "b*", label="lower slab envelop")
ax.plot(ps1[:, 3], depths1/1e3, "y*", label="upper slab envelop")
ax.invert_yaxis()
ax.set_xlabel("Dynamic Pressure (Pa)")
ax.set_ylabel("Depth (km)")
ax.legend()

fig, ax = plt.subplots()
ax.plot(thetas[:, 0]*180.0/np.pi, depths1/1e3, "b*", label="dip angle")
ax.invert_yaxis()
ax.set_xlabel("Dip Angle")
ax.set_ylabel("Depth (km)")
ax.legend()